In [1]:
import pandas as pd
import numpy as np

In [2]:
train  = pd.read_csv('train.csv')
dup = train.pop('is_duplicate')

In [3]:
train.head()


,id,qid1,qid2,question1,question2
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?


In [4]:
#cleaning the text
import nltk
import re, string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def clean_text(content):
    if type(content) == str:
        #lowercasing the text
        text = content.lower()
        # Removing non ASCII chars
        text = re.sub(r'[^\x00-\x7f]',r' ',text)
        # Removing (replacing with empty spaces actually) all the punctuations
        text = re.sub("["+string.punctuation+"]", " ", text)
        #splitting into words
        words=text.split()
        #removing the stop words in the text
        stop_word=set(stopwords.words('english'))
        words=list(word for word in words if not word in stop_word)
        words=[word for word in words if len(word)>1 ]
        words=[WordNetLemmatizer().lemmatize(word) for word in words]
        return ( " ".join(words) )
    else:
        return ""

In [8]:
train.question1 = train.question1.map(clean_text)
train.question2 = train.question2.map(clean_text)

In [11]:
def tokenize(content):
    return content.split()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenize, analyzer='word', max_df=1.0, min_df=1)
cosine_vals = []

for i in train.id:
    try:
        tfidf_matrix = tfidf_vectorizer.fit_transform([train.loc[i]['question1'], train.loc[i]['question2']])
        cosine_vals.append(cosine_similarity(tfidf_matrix[0:1], tfidf_matrix)[0][1])
    except:
        cosine_vals.append(0)

In [22]:
y_pred=cosine_vals

In [25]:
df = pd.DataFrame({'col':cosine_vals})

In [30]:
df.head()

,col
0,0.895532
1,0.410995
2,0.225765
3,0.000000
4,0.168368


In [32]:
dup.head()

0    0
1    0
2    0
3    0
4    0
Name: is_duplicate, dtype: int64